
useful:

- https://magenta.tensorflow.org/svg-vae
- github: https://github.com/magenta/magenta/tree/master/magenta/models/svg_vae
- notebook: https://github.com/magenta/magenta-demos/blob/master/colab-notebooks/vae_svg_decoding.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt

In [3]:
SEED = 42
records_dir = pathlib.Path("./tfrs")
file_paths = [str(fp) for fp in list(records_dir.glob("*"))]

In [4]:
filepath_dataset = tf.data.Dataset.list_files(file_paths, seed=SEED)

In [5]:
full_dataset = filepath_dataset.interleave(
        lambda filepath: tf.data.TFRecordDataset(filepath),
        cycle_length=tf.data.experimental.AUTOTUNE,
    )

In [6]:
# for raw_record in full_dataset.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())
#     print(example)

In [7]:
feature_description = {
    'rendered': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'seq_len': tf.io.FixedLenFeature([], tf.int64),
    'class': tf.io.FixedLenFeature([], tf.int64),
    'binary_fp': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
    'sequence': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

In [8]:
parsed_dataset = full_dataset.map(_parse_function)

In [9]:
for a in parsed_dataset.take(2):
    rendered = np.reshape(a['rendered'].numpy(), (64,64))
    sequence = np.reshape(a['sequence'].numpy(), (51,10))
    classx = np.reshape(a['class'].numpy(), (1))
    seq_len = np.reshape(a['seq_len'].numpy(), (1))
    binary_fp = a['binary_fp'].numpy()
#     plt.imshow(rendered)
#     plt.show()